- Homework2
- **This homework focuses on training and evaluating prediction models for a particular problem and dataset.**
- **In this analysis, we focus on using the data collected by CDC to build a data analytics solution for death risk prediction.**
- The goal in this homework is to work with the data to build and evaluate prediction models that capture the relationship between the descriptive features and the target feature **death_yn**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt